In [4]:
%load_ext autoreload
%autoreload 2
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-21T03:30:00"
event_start = "2010-01-20"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / "runs/gridrad_demo"

In [23]:
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)

lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    global_flow_margin=70,
    unique_global_flow=False,
)

option.check_options(track_options)
option.save_track_options(track_options, options_directory=options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options = None

2024-10-11 11:52:40,482 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/data.yml
2024-10-11 11:52:40,512 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-11 11:52:40,514 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/grid.yml
2024-10-11 11:52:40,522 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/track.yml


In [15]:
times = data.utils.generate_times(data_options["gridrad"])
track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_parent
)

2024-10-11 00:12:25,917 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo.
2024-10-11 00:12:25,919 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-11 00:12:25,962 - thor.data.utils - INFO - Initiating download of https://data.rda.ucar.edu/d841006/volumes/2010/20100120/nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-11 00:12:25,964 - thor.data.utils - INFO - Sending HTTP request to https://data.rda.ucar.edu/d841006/volumes/2010/20100120/nexrad_3d_v4_2_20100120T180000Z.nc.
10.2MiB [00:03, 2.67MiB/s]
2024-10-11 00:12:30,971 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-11 00:12:30,973 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-11 00:12:30,973 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-11 00:12:30,974 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-11 00:12:31

KeyboardInterrupt: 

In [24]:
with log.logging_listener():
    with multiprocessing.Pool(initializer=parallel.initialize_process) as pool:
        results = []
        for i, time_interval in enumerate(intervals):
            args = [i, time_interval, data_options.copy(), grid_options.copy()]
            args += [track_options.copy(), visualize_options] 
            args += [output_parent, "gridrad"]
            args = tuple(args)
            results.append(pool.apply_async(parallel.track_interval, args))
        pool.close()
        pool.join()
        parallel.check_results(results)

2024-10-11 11:53:05,070 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/data.yml
2024-10-11 11:53:05,070 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_1/options/data.yml
2024-10-11 11:53:05,071 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_4/options/data.yml
2024-10-11 11:53:05,072 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_2/options/data.yml
2024-10-11 11:53:05,073 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_3/options/data.yml
2024-10-11 11:53:05,101 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_4/options/grid.yml
2024-10-11 11:53:05,102 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/grid.yml
2024-10-11 11:53:05,104 - thor.option - DEBUG - Saving 

In [27]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-10-11 12:01:13,051 - thor.parallel - INFO - Stitching all attribute, mask and record files.


KeyError: 'track'

In [28]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-11 12:01:14,285 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.csv
2024-10-11 12:01:14,291 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.yml
2024-10-11 12:01:14,454 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.csv
2024-10-11 12:01:14,459 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.yml
2024-10-11 12:01:14,537 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.csv
2024-10-11 12:01:14,541 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.yml


stratiform_offset inflow  \
time                universal_id                            
2010-01-20 18:00:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:10:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:20:00 1                      leading  right   
...                                            ...    ...   
2010-01-21 03:10:00 8                        right  front   
                    20                       right  front   
                    25                    trailing  front   
2010-01-21 03:20:00 8                      leading  front   
                    20                       right  front   

                                 relative_stratiform_offset  \
time                universal_id                              
2010-01-20 18:00:00 1                                  left   
                    2                                  left   
2010-01-20 18:10:00 1                                  left   
                    2                                  left   
2010-01-20 18:20:00 1                                  left   
...                                                     ...   
2010-01-21 03:10:00 8                               leading   
                    20                                right   
                    25                             trailing   
2010-01-21 03:20:00 8                               leading   
                    20                                right   

                                                 tilt          propagation  
time                universal_id                                            
2010-01-20 18:00:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:10:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:20:00 1                      down-shear           down-shear  
...                                               ...                  ...  
2010-01-21 03:10:00 8                      down-shear           down-shear  
                    20            shear-perpendicular           down-shear  
                    25                       up-shear           down-shear  
2010-01-21 03:20:00 8                      down-shear           down-shear  
                    20            shear-perpendicular           down-shear  

[248 rows x 5 columns]

In [6]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2010-01-20T18:00")
end_time = np.datetime64(np.datetime64("2010-01-21T03:30"))
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 5400, "by_date": False}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-11 12:04:26,448 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-11 12:04:26,598 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-11 12:04:27,180 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-11 12:04:27,223 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-11 12:04:27,447 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-11 12:04:27,680 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-11 12:04:28,010 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-11 12:04:28,114 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-11 12:04:28,116 - thor.data.gridrad - DEBUG - Got grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-11 12:04:28,116 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time